In [14]:
from gruut import sentences

def english_cleaners3(text):
    """Pipeline for English text, including phonemization + punctuation"""
    text = text.replace("à", "a")
    phonemizer = lambda s: [
        [word.text] if word.is_major_break or word.is_minor_break else word.phonemes
        for words in sentences(s)
        for word in words
    ]

    phonemes = phonemizer(text)
    phonemes = [p for p in phonemes if p != None]
    phonemes = "# #".join(["#".join(phn) for phn in phonemes])
    return phonemes

In [13]:
english_cleaners3("bric-à-brac")

[['b', 'ɹ', 'ˈɪ', 'k'], ['ə'], ['b', 'ɹ', 'ˈæ', 'k']]


'b#ɹ#ˈɪ#k# #ə# #b#ɹ#ˈæ#k'

In [15]:
import pandas as pd
import re
from tqdm.contrib.concurrent import process_map

# df = pd.read_csv("/root/hifi-gan/en_au_dean2zak/training.txt", sep="|", header=None, names=["id", "phonemes"])
df = pd.read_csv("/root/pflowtts_pytorch/data/LJSpeech-1.1/ljs_audio_text_train_filelist.txt", sep="|", header=None, names=["id", "phonemes"])
df["phonemes"] = df["phonemes"].apply(lambda x: x.replace("_1", "").replace("_2", "").replace(". . .", "."))
# collapse multiple whitespace with regex
df["phonemes"] = df["phonemes"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
phonemes = process_map(english_cleaners3, df["phonemes"])
df["phonemes"] = phonemes
# df["id"] = df["id"].apply(lambda x: "/root/pflowtts_pytorch/en_au_dean2zak/audio/" + x + ".wav")
df["id"] = df["id"].apply(lambda x: "/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/" + x.split("/")[-1])
# df.to_csv("/root/hifi-gan/en_au_dean2zak/filelists/training.txt", sep="|", header=None, index=None)
df.to_csv("/root/pflowtts_pytorch/data/LJSpeech-1.1/filelists/ljs_audio_text_train_filelist.txt", sep="|", header=None, index=None)
df

/tmp/ipykernel_1637433/892393068.py:11: TqdmWarning: Iterable length 11917 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  phonemes = process_map(english_cleaners3, df["phonemes"])


  0%|          | 0/11917 [00:00<?, ?it/s]

,id,phonemes
0,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ˈɪ#t# #h#ˈæ#z# #j#ˈu#s#t# #ˈʌ#ð#ɚ# #t#ɹ#ˈɛ#ʒ#ɚ...
1,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,t#ə# #ə#v#ˈeɪ#l# #h#ɪ#m#s#ˈɛ#l#f# #ə#v# #h#ˈɪ#...
2,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ɔ#l#ð#ˈoʊ# #t͡ʃ#ˈi#f# #ɹ#ˈoʊ#l#i# #d#ˈʌ#z# #n#...
3,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ð#ə# #θ#ɹ#ˈi# #ˈɔ#f#ɪ#s#ɚ#z# #k#ə#n#f#ˈɚ#m# #ð...
4,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ð#ə# #t#ɹ#ˈaɪ#d# #ˈæ#n#d# #ð#ə# #ʌ#n#t#ɹ#ˈaɪ#d...
...,...,...
11912,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ˈɪ#n# #ə#n#ˈʌ#ð#ɚ# #p#ˈɑ#ɹ#t# #s#ˈɛ#v#ə#n# #p#...
11913,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,"h#ˌaʊ#ˈɛ#v#ɚ# #,# #ð#ə# #n#ˈaɪ#n#t#ˈi#n# #s#ˈɪ..."
11914,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,ˈɪ#t# #ˈɪ#z# #ð#ə# #ə#m#ˈɛ#ɹ#ɪ#k#ə#n# #p#ˈi#p#...
11915,/root/pflowtts_pytorch/data/LJSpeech-1.1/wavs/...,"h#ˈæ#n#d#ɹ#ˌaɪ#t#ɪ#ŋ# #ˈɛ#k#s#p#ɚ#t#s# #,# #ˈæ..."


In [1]:
from pydub import AudioSegment

def resample(path):
    sound = AudioSegment.from_file(path)
    sound.export(path.replace("wavs/", "wavs_44/"), format="wav", parameters=["-ar", "44100"])

In [5]:
from glob import glob
from tqdm.contrib.concurrent import process_map

audios = sorted(glob("/root/LJSpeech-1.1/wavs/*.wav"))
_ = process_map(resample, audios)

/tmp/ipykernel_1637433/362126750.py:5: TqdmWarning: Iterable length 13100 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  _ = process_map(resample, audios)


  0%|          | 0/13100 [00:00<?, ?it/s]